In [6]:
import sys, sagemaker, boto3
!{sys.executable} -m pip install -qU "sagemaker>=2.11.0"
print(sagemaker.__version__)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2.209.0


In [7]:
import torch
print(torch.__version__)

import transformers
print(transformers.__version__)

!python --version

2.1.0
4.38.2
Python 3.10.13


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import json
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session

sess = Session()
role = get_execution_role()
print(f'Current session is : {sess}, \nCurrent role is : {role}')

# check default bucket
sess.default_bucket()

In [ ]:
s3_client = boto3.client('s3')
response = s3_client.list_buckets()

# 버킷 이름 출력
for bucket in response['Buckets']:
    print(bucket['Name'])

In [10]:
bucket = "error-correction"

## S3 to SageMakerNotebook
    - S3에 디아카이빙된 파일들을 올린 상태
    - 다운로드해온 후 해당 노트북 인스턴스에서 inference.py(handler.py) 코드 수정

In [22]:
import boto3

# S3 클라이언트 초기화
s3 = boto3.client('s3')

# S3 버킷 및 폴더 이름 지정
bucket = "error-correction"
prefix = 'flan_t5_xlarge/'  # 폴더 이름

# 지정된 폴더 내의 모든 파일 나열
def list_files_in_folder(bucket, prefix):
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    files = []

    # 폴더 내의 모든 파일을 나열
    for item in response.get('Contents', []):
        files.append(item['Key'])

    return files

files = list_files_in_folder(bucket, prefix)
files = files[1:] # 첫번째 의미없는 폴더 제거

for file in files:
    print(file)

flan_t5_xlarge/config.json
flan_t5_xlarge/err_corr_generation.ipynb
flan_t5_xlarge/generation_config.json
flan_t5_xlarge/handler.py
flan_t5_xlarge/postprocess.py
flan_t5_xlarge/pytorch_model.bin
flan_t5_xlarge/special_tokens_map.json
flan_t5_xlarge/tokenizer.json
flan_t5_xlarge/tokenizer_config.json


In [23]:
import os

# 파일 다운로드 함수
def download_file(bucket, s3_file_path, local_save_path):
    # 필요한 경우 로컬 저장 경로의 디렉토리 생성
    os.makedirs(os.path.dirname(local_save_path), exist_ok=True)
    
    # 파일 다운로드
    s3.download_file(bucket, s3_file_path, local_save_path)
    print(f'Downloaded {s3_file_path} to {local_save_path}')

# 로컬 저장 경로 지정 (예시)
local_folder = 'flan_t5_xlarge'

# 파일 다운로드 실행
for file_key in files:
    local_file_path = os.path.join(local_folder, os.path.basename(file_key))
    download_file(bucket, file_key, local_file_path)

Downloaded flan_t5_xlarge/config.json to flan_t5_xlarge/config.json
Downloaded flan_t5_xlarge/err_corr_generation.ipynb to flan_t5_xlarge/err_corr_generation.ipynb
Downloaded flan_t5_xlarge/generation_config.json to flan_t5_xlarge/generation_config.json
Downloaded flan_t5_xlarge/handler.py to flan_t5_xlarge/handler.py
Downloaded flan_t5_xlarge/postprocess.py to flan_t5_xlarge/postprocess.py
Downloaded flan_t5_xlarge/pytorch_model.bin to flan_t5_xlarge/pytorch_model.bin
Downloaded flan_t5_xlarge/special_tokens_map.json to flan_t5_xlarge/special_tokens_map.json
Downloaded flan_t5_xlarge/tokenizer.json to flan_t5_xlarge/tokenizer.json
Downloaded flan_t5_xlarge/tokenizer_config.json to flan_t5_xlarge/tokenizer_config.json


## inference test

In [1]:
from flan_t5_xl.inference import model_fn, input_fn, predict_fn, output_fn
import json
import torch
import os
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [2]:
# model 저장 경로
model_dir = 'flan_t5_xl'

model_handler = model_fn(model_dir)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# 추론을 위한 입력 데이터를 준비합니다.
dialogue = [
    "Tutor: Thanks, love. But let's focus on planning our anniversary first. Any suggestions? Student: you very nice Tutor: Aww, you're sweet. Now, let's talk about our anniversary dinner. Do you have any cuisine in mind? [Improve]Student: How about a picnic?",
    "Tutor: Sure, how about this navy blue suit with a thin light-blue stripe? [Improve]Student: This stripe is too small. Is there anything bigger?"
]

# dialogue를 각 항목별로 JSON 형식으로 변환합니다. 이 예시에서는 각 대화를 'data' 키를 가진 딕셔너리로 만듭니다.
request_body = json.dumps([{"data": text} for text in dialogue])
print('resquest_body : ', request_body)

request_content_type = 'application/json'

# 입력 데이터를 전처리합니다.
input_data = input_fn(request_body, request_content_type)
# print('input data : ', input_data)

# 모델 추론을 실행합니다.
prediction = predict_fn(input_data, model_handler)
# print('prediction : ', prediction)

# 추론 결과를 처리합니다.
output = output_fn(prediction, 'application/json')

# 추론 결과를 출력합니다.
print(output)

resquest_body :  [{"data": "Tutor: Thanks, love. But let's focus on planning our anniversary first. Any suggestions? Student: you very nice Tutor: Aww, you're sweet. Now, let's talk about our anniversary dinner. Do you have any cuisine in mind? [Improve]Student: How about a picnic?"}, {"data": "Tutor: Sure, how about this navy blue suit with a thin light-blue stripe? [Improve]Student: This stripe is too small. Is there anything bigger?"}]
input_data [{'data': "Tutor: Thanks, love. But let's focus on planning our anniversary first. Any suggestions? Student: you very nice Tutor: Aww, you're sweet. Now, let's talk about our anniversary dinner. Do you have any cuisine in mind? [Improve]Student: How about a picnic?"}, {'data': 'Tutor: Sure, how about this navy blue suit with a thin light-blue stripe? [Improve]Student: This stripe is too small. Is there anything bigger?'}]


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


utterances :  ['How about a picnic?', 'This stripe is too small. Is there anything bigger?']
utter :  how about a picnic?
corr :  how about a picnic?
utter :  this stripe is too small is there anything bigger?
corr :  this stripe is too small is there anything bigger?
{"generated_text": ["how about a picnic?", "this stripe is too small is there anything bigger?"]}


## model.tar.gz 파일 만들기

In [ ]:
# tar -czvf model.tar.gz config.json generation_config.json tokenizer.json tokenizer_config.json special_tokens_map.json pytorch_model.bin inference.py

## s3에 model.tar.gz 업로드

In [12]:
import boto3
import os

# AWS 접속 정보
ACCESS_KEY_ID = 
ACCESS_SECRET_KEY = 
BUCKET_NAME = 'error-correction'

# 업로드할 파일이 위치한 폴더
FOLDER_PATH = 'flan_t5_xl'

# 업로드할 파일 리스트
upload_files = ['model.tar.gz']

def upload_files_to_s3(folder_path, upload_files):
    # S3 클라이언트 생성
    s3_client = boto3.client('s3',
                             aws_access_key_id=ACCESS_KEY_ID,
                             aws_secret_access_key=ACCESS_SECRET_KEY)

    # 지정된 파일만 업로드
    for filename in upload_files:
        local_file_path = os.path.join(folder_path, filename)
        if os.path.exists(local_file_path):  # 파일 존재 여부 확인
            print(f"Uploading {filename}...")
            s3_target_path = 'flan_t5_xlarge/' + filename  # S3 내의 대상 경로 설정

            # 파일 업로드
            s3_client.upload_file(local_file_path, BUCKET_NAME, s3_target_path)
            print(f'Uploaded {filename} to {s3_target_path}')
        else:
            print(f"{filename} does not exist in {folder_path}")

# 함수 호출
upload_files_to_s3(FOLDER_PATH, upload_files)

Uploading model.tar.gz...
Uploaded model.tar.gz to flan_t5_xlarge/model.tar.gz


## SageMaker endpoint

In [ ]:
import json
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session

sess = Session()
role = get_execution_role()
print(f'Current session is : {sess}, \nCurrent role is : {role}')

# check default bucket
sess.default_bucket()

In [7]:
import os
import time
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer
from sagemaker.pytorch.model import PyTorchModel

In [8]:
%%time
import time

model_path = "s3://error-correction/flan_t5_xlarge/model.tar.gz"
endpoint_name = "endpoint-pytorch-{}".format(int(time.time()))

model = PyTorchModel(model_data=model_path,
                     role=role,
                     entry_point='inference.py', 
                     source_dir='src',
                     framework_version='2.0.1',
                     py_version='py310')

CPU times: user 697 µs, sys: 0 ns, total: 697 µs
Wall time: 494 µs


In [ ]:
#!tar -czvf model.tar.gz inference.py config.json generation_config.json tokenizer.json tokenizer_config.json special_tokens_map.json pytorch_model.bin

In [9]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.2xlarge',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

-----------!

In [21]:
# !pip freeze > requirements.txt

In [27]:
input = [{
   "data": "Tutor: Thanks, love. But let's focus on planning our anniversary first. Any suggestions? Student: you very nice Tutor: Aww, you're sweet. Now, let's talk about our anniversary dinner. Do you have any cuisine in mind? [Improve]Student: How about a picnic?"
}]

In [28]:
predictor.predict(input)

{'generated_text': ['how about a picnic?']}

In [30]:
import boto3
import json
import time

endpoint_name = predictor.endpoint_name
client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = [{'data': "i had symptom"}]
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

# API 호출 전 시간 기록
start_time = time.time()

# API 호출
response = client.invoke_endpoint(
EndpointName=endpoint_name,
ContentType=content_type,
Body=payload)

# API 호출 후 시간 기록
end_time = time.time()

# 호출 속도 계산
duration = end_time - start_time

# 출력
print(f"API call duration: {duration:.2f} seconds")

# 호출 결과
result = response['Body'].read().decode('utf-8')
result

API call duration: 0.26 seconds


'{"generated_text": ["i had symptom"]}'

In [37]:
# concurrent calls 

from concurrent.futures import ThreadPoolExecutor
client = boto3.client('sagemaker-runtime')

# request function
def send_request(request_data):
    start = time.time()  
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,  
        ContentType=content_type,
        Body=json.dumps(request_data)
    )
    end = time.time()  
    duration = end - start  
    return start, duration, response['Body'].read().decode('utf-8')


# 동시 요청의 수
num_requests = 100


# ThreadPoolExecutor를 사용하여 동시 요청 수행
with ThreadPoolExecutor(max_workers=num_requests) as executor:
    futures = [executor.submit(send_request, data) for _ in range(num_requests)]

    # 결과 출력
    for future in futures:
        start_time, duration, result = future.result()
        print(f"Request start time: {start_time:.2f}, Duration: {duration:.2f} seconds, Result: {result}")

# 전체 요청 완료 후의 현재 시간
overall_end_time = time.time()

# 시작 시간 중 가장 빠른 시간 찾기
start_times = [future.result()[0] for future in futures]
overall_start_time = min(start_times)

# 전체 실행 시간 출력
print(f"Total duration for all requests: {overall_end_time - overall_start_time:.2f} seconds")

Request start time: 1709801305.74, Duration: 0.94 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.74, Duration: 0.52 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.75, Duration: 1.14 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.75, Duration: 0.31 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.75, Duration: 1.76 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.75, Duration: 0.72 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.75, Duration: 1.55 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.75, Duration: 3.42 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.75, Duration: 1.97 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709801305.75, Duration: 2.59 secon

In [41]:
# concurrent calls 

from concurrent.futures import ThreadPoolExecutor
client = boto3.client('sagemaker-runtime')

# request function
def send_request(request_data):
    start = time.time()  
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,  
        ContentType=content_type,
        Body=json.dumps(request_data)
    )
    end = time.time()  
    duration = end - start  
    return start, duration, response['Body'].read().decode('utf-8')


# 동시 요청의 수
num_requests = 150


# ThreadPoolExecutor를 사용하여 동시 요청 수행
with ThreadPoolExecutor(max_workers=num_requests) as executor:
    futures = [executor.submit(send_request, data) for _ in range(num_requests)]

    # 결과 출력
    for future in futures:
        start_time, duration, result = future.result()
        print(f"Request start time: {start_time:.2f}, Duration: {duration:.2f} seconds, Result: {result}")

# 전체 요청 완료 후의 현재 시간
overall_end_time = time.time()

# 시작 시간 중 가장 빠른 시간 찾기
start_times = [future.result()[0] for future in futures]
overall_start_time = min(start_times)

# 전체 실행 시간 출력
print(f"Total duration for all requests: {overall_end_time - overall_start_time:.2f} seconds")

Request start time: 1709803115.98, Duration: 1.54 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.98, Duration: 0.72 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.99, Duration: 0.27 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.99, Duration: 1.13 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.99, Duration: 0.51 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.99, Duration: 4.00 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.99, Duration: 1.74 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.99, Duration: 2.15 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.99, Duration: 1.33 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803115.99, Duration: 1.94 secon

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (503) from primary with message "{
  "code": 503,
  "type": "ServiceUnavailableException",
  "message": "Model \"model\" has no worker to serve inference request. Please use scale workers API to add workers."
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2024-03-07-07-39-18-144 in account 699188727212 for more information.

In [40]:
# concurrent calls 

from concurrent.futures import ThreadPoolExecutor
client = boto3.client('sagemaker-runtime')

# request function
def send_request(request_data):
    start = time.time()  
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,  
        ContentType=content_type,
        Body=json.dumps(request_data)
    )
    end = time.time()  
    duration = end - start  
    return start, duration, response['Body'].read().decode('utf-8')


# 동시 요청의 수
num_requests = 100


# ThreadPoolExecutor를 사용하여 동시 요청 수행
with ThreadPoolExecutor(max_workers=num_requests) as executor:
    futures = [executor.submit(send_request, data) for _ in range(num_requests)]

    # 결과 출력
    for future in futures:
        start_time, duration, result = future.result()
        print(f"Request start time: {start_time:.2f}, Duration: {duration:.2f} seconds, Result: {result}")

# 전체 요청 완료 후의 현재 시간
overall_end_time = time.time()

# 시작 시간 중 가장 빠른 시간 찾기
start_times = [future.result()[0] for future in futures]
overall_start_time = min(start_times)

# 전체 실행 시간 출력
print(f"Total duration for all requests: {overall_end_time - overall_start_time:.2f} seconds")

Request start time: 1709803060.77, Duration: 0.29 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.77, Duration: 0.50 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.77, Duration: 1.73 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.77, Duration: 4.20 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.77, Duration: 0.91 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.77, Duration: 4.40 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.78, Duration: 2.95 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.78, Duration: 0.70 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.78, Duration: 1.52 seconds, Result: {"generated_text": ["i had symptom"]}
Request start time: 1709803060.78, Duration: 3.98 secon